In [36]:
# Import necessary modules and functions

import pandas as pd
import seaborn as sns
import numpy as np
import nltk
import datetime

import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 16, 8

from scipy.stats import norm
from scipy import stats

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


import warnings
warnings.filterwarnings('ignore')

kaggledf = pd.read_csv('KS_train_data.csv', delimiter = ',')

In [37]:
test_columns = ["project_id", "blurb", "category", "country", "created_at", "currency", "deadline", "fx_rate", "goal", "launched_at", "name", "staff_pick", "location", "subcategory", "project_url"]

#drop colums that are not in test_set + reward URL
kaggledf = kaggledf.drop(columns=[col for col in kaggledf if col not in test_columns])

In [38]:
#calculate seconds from UNIX timestamp
kaggledf["seconds"] = kaggledf["deadline"] - kaggledf["created_at"]

In [39]:
#convert all goals to USD
def convert_usd(row):
    return row["goal"] * row["fx_rate"]
    
    
kaggledf["usd_goal"] = kaggledf.apply(lambda row: convert_usd(row), axis = 1)


#delete unnessesary colums
currency_col = ["fx_rate", "currency", "goal"]
kaggledf = kaggledf.drop(columns=[col for col in kaggledf if col in currency_col])

In [40]:
#strip url of unnessesary stuff
def stripper(row): #XD
    raw = row["project_url"].split("/")[-1].split("-")
    return " ".join(raw)

kaggledf["stripped_url"] = kaggledf.apply(lambda row: stripper(row), axis = 1)

#delete original url column
kaggledf = kaggledf.drop(columns = ["project_url", "name"])

In [41]:
#categorical to numerical data
dummys=[
    dummy_category := pd.get_dummies(kaggledf['category'], prefix = "category_"),
    dummy_subcategory := pd.get_dummies(kaggledf['subcategory'], prefix = "subcategory_"),
    dummy_country := pd.get_dummies(kaggledf['country'], prefix = "country_")]

for dummy in dummys:
    kaggledf = pd.merge(
        left=kaggledf,
        right=dummy,
        left_index=True,
        right_index=True,
    )
    
kaggledf = kaggledf.drop(columns = ["category","subcategory","country"])

In [44]:

#cange staffpick to numerical
kaggledf["staff_pick"] = kaggledf["staff_pick"].apply(lambda x: 1 if x == True else 0)

In [45]:
pd.set_option("display.max_rows", 10, "display.max_columns", None)
#categorical ranges for goal and pledged, sentiment vader (numerical & categorical)  
#columns to be removed: url's, location, id,
kaggledf.dropna(inplace=True)
kaggledf

,project_id,blurb,created_at,deadline,launched_at,staff_pick,location,seconds,usd_goal,stripped_url,category__art,category__comics,category__crafts,category__dance,category__design,category__fashion,category__film & video,category__food,category__games,category__journalism,category__music,category__photography,category__publishing,category__technology,category__theater,subcategory__3d printing,subcategory__academic,subcategory__accessories,subcategory__action,subcategory__animals,subcategory__animation,subcategory__anthologies,subcategory__apparel,subcategory__apps,subcategory__architecture,subcategory__art,subcategory__art books,subcategory__audio,subcategory__bacon,subcategory__blues,subcategory__calendars,subcategory__camera equipment,subcategory__candles,subcategory__ceramics,subcategory__children's books,subcategory__childrenswear,subcategory__chiptune,subcategory__civic design,subcategory__classical music,subcategory__comedy,subcategory__comic books,subcategory__comics,subcategory__community gardens,subcategory__conceptual art,subcategory__cookbooks,subcategory__country & folk,subcategory__couture,subcategory__crafts,subcategory__crochet,subcategory__dance,subcategory__design,subcategory__digital art,subcategory__diy,subcategory__diy electronics,subcategory__documentary,subcategory__drama,subcategory__drinks,subcategory__electronic music,subcategory__embroidery,subcategory__events,subcategory__experimental,subcategory__fabrication tools,subcategory__faith,subcategory__family,subcategory__fantasy,subcategory__farmer's markets,subcategory__farms,subcategory__festivals,subcategory__fiction,subcategory__film & video,subcategory__fine art,subcategory__flight,subcategory__food,subcategory__food trucks,subcategory__footwear,subcategory__gadgets,subcategory__games,subcategory__gaming hardware,subcategory__glass,subcategory__graphic design,subcategory__graphic novels,subcategory__hardware,subcategory__hip-hop,subcategory__horror,subcategory__illustration,subcategory__immersive,subcategory__indie rock,subcategory__installations,subcategory__interactive design,subcategory__jazz,subcategory__jewelry,subcategory__journalism,subcategory__kids,subcategory__knitting,subcategory__latin,subcategory__letterpress,subcategory__literary journals,subcategory__literary spaces,subcategory__live games,subcategory__makerspaces,subcategory__metal,subcategory__mixed media,subcategory__mobile games,subcategory__movie theaters,subcategory__music,subcategory__music videos,subcategory__musical,subcategory__narrative film,subcategory__nature,subcategory__nonfiction,subcategory__painting,subcategory__people,subcategory__performance art,subcategory__performances,subcategory__periodicals,subcategory__pet fashion,subcategory__photo,subcategory__photobooks,subcategory__photography,subcategory__places,subcategory__playing cards,subcategory__plays,subcategory__poetry,subcategory__pop,subcategory__pottery,subcategory__print,subcategory__printing,subcategory__product design,subcategory__public art,subcategory__publishing,subcategory__punk,subcategory__puzzles,subcategory__quilts,subcategory__r&b,subcategory__radio & podcasts,subcategory__ready-to-wear,subcategory__residencies,subcategory__restaurants,subcategory__robots,subcategory__rock,subcategory__romance,subcategory__science fiction,subcategory__sculpture,subcategory__shorts,subcategory__small batch,subcategory__software,subcategory__sound,subcategory__space exploration,subcategory__spaces,subcategory__stationery,subcategory__tabletop games,subcategory__taxidermy,subcategory__technology,subcategory__television,subcategory__textiles,subcategory__theater,subcategory__thrillers,subcategory__translations,subcategory__typography,subcategory__vegan,subcategory__video,subcategory__video art,subcategory__video games,subcategory__wearables,subcategory__weaving,subcategory__web,subcategory__webcomics,subcategory__webseries,subcategory__woodworking,subcategory__workshops,subcategory__world music,subcategory__young adult